# Cabin

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier, Pool

In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
le = LabelEncoder()

<IPython.core.display.Javascript object>

In [4]:
df = pd.read_csv("../../data/titanic/prepare.csv").set_index("PassengerId")
df

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,...,FullName,Surname,Title,FamilySize,TicketSeries,NameSeries,SurnameSeries,TitleSeries,IsTest,isAgePreds
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,0.0,3,Owen Harris,1,22.000000,1,0,A/5 21171,-0.503595,NaN,...,"Braund, Mr. Owen Harris",Braund,Mr,2,2,855,100,12,0,0
2,1.0,1,John Bradley (Florence Briggs Thayer),0,38.000000,1,0,PC 17599,0.734503,C85,...,"Cumings, Mrs. John Bradley (Florence Briggs Th...",Cumings,Mrs,2,17,601,182,13,0,0
3,1.0,3,Laina,0,26.000000,0,0,STON/O2. 3101282,-0.490544,NaN,...,"Heikkinen, Miss. Laina",Heikkinen,Miss,1,34,690,329,9,0,0
4,1.0,1,Jacques Heath (Lily May Peel),0,35.000000,1,0,113803,0.382925,C123,...,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",Futrelle,Mrs,2,40,541,267,13,0,0
5,0.0,3,William Henry,1,35.000000,0,0,373450,-0.488127,NaN,...,"Allen, Mr. William Henry",Allen,Mr,1,40,1102,15,12,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,NaN,3,Woolf,1,26.994279,0,0,A.5. 3236,-0.488127,NaN,...,"Spector, Mr. Woolf",Spector,Mr,1,5,1119,753,12,1,1
1306,NaN,1,Fermina,0,39.000000,0,0,PC 17758,1.461829,C105,...,"Oliva y Ocana, Dona. Fermina",Oliva y Ocana,Dona,1,17,366,593,3,1,0
1307,NaN,3,Simon Sivertsen,1,38.500000,0,0,SOTON/O.Q. 3101262,-0.503595,NaN,...,"Saether, Mr. Simon Sivertsen",Saether,Mr,1,31,973,699,12,1,0


<IPython.core.display.Javascript object>

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 1 to 1309
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Survived       891 non-null    float64
 1   Pclass         1309 non-null   int64  
 2   Name           1309 non-null   object 
 3   Sex            1309 non-null   int64  
 4   Age            1309 non-null   float64
 5   SibSp          1309 non-null   int64  
 6   Parch          1309 non-null   int64  
 7   Ticket         1309 non-null   object 
 8   Fare           1309 non-null   float64
 9   Cabin          295 non-null    object 
 10  Embarked       1309 non-null   int64  
 11  FullName       1309 non-null   object 
 12  Surname        1309 non-null   object 
 13  Title          1309 non-null   object 
 14  FamilySize     1309 non-null   int64  
 15  TicketSeries   1309 non-null   int64  
 16  NameSeries     1309 non-null   int64  
 17  SurnameSeries  1309 non-null   int64  
 18  TitleSer

<IPython.core.display.Javascript object>

In [6]:
X = df.drop(
    [
        "Survived",
        "Name",
        "Ticket",
        "Cabin",
        "FullName",
        "Surname",
        "Title",
        "IsTest",
        "isAgePreds",
    ],
    axis=1,
)
y = df[["Age"]]
X

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,TicketSeries,NameSeries,SurnameSeries,TitleSeries
PassengerId,,,,,,,,,,,,
1,3,1,22.000000,1,0,-0.503595,2,2,2,855,100,12
2,1,0,38.000000,1,0,0.734503,0,2,17,601,182,13
3,3,0,26.000000,0,0,-0.490544,2,1,34,690,329,9
4,1,0,35.000000,1,0,0.382925,2,2,40,541,267,13
5,3,1,35.000000,0,0,-0.488127,2,1,40,1102,15,12
...,...,...,...,...,...,...,...,...,...,...,...,...
1305,3,1,26.994279,0,0,-0.488127,2,1,5,1119,753,12
1306,1,0,39.000000,0,0,1.461829,0,1,17,366,593,3
1307,3,1,38.500000,0,0,-0.503595,2,1,31,973,699,12


<IPython.core.display.Javascript object>

In [7]:
train_index = df[~df["Cabin"].isna()].index
train_index

Int64Index([   2,    4,    7,   11,   12,   22,   24,   28,   32,   53,
            ...
            1282, 1283, 1287, 1289, 1292, 1296, 1297, 1299, 1303, 1306],
           dtype='int64', name='PassengerId', length=295)

<IPython.core.display.Javascript object>

In [8]:
X_train = X[X.index.isin(train_index)]
X_test = X[~X.index.isin(train_index)]

X_train.shape, X_test.shape

((295, 12), (1014, 12))

<IPython.core.display.Javascript object>

## Prepare `y`

In [9]:
df["CabinSeries"] = df["Cabin"].str[0]

df["CabinSeries"] = df["CabinSeries"].astype("category").cat.codes
df.loc[df["CabinSeries"] == -1, "CabinSeries"] = np.nan

df["CabinSeries"].value_counts(dropna=False)

NaN    1014
2.0      94
1.0      65
3.0      46
4.0      41
0.0      22
5.0      21
6.0       5
7.0       1
Name: CabinSeries, dtype: int64

<IPython.core.display.Javascript object>

In [10]:
y_train = df[df.index.isin(train_index)][["CabinSeries"]]
y_train

,CabinSeries
PassengerId,
2,2.0
4,2.0
7,4.0
11,6.0
12,2.0
...,...
1296,3.0
1297,3.0
1299,2.0


<IPython.core.display.Javascript object>

# Train

In [11]:
model = CatBoostClassifier(loss_function="MultiClass")

<IPython.core.display.Javascript object>

In [12]:
model.fit(Pool(X_train, y_train), verbose=False, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

<IPython.core.display.Javascript object>

# Save

In [13]:
y_preds = model.predict(X_test).ravel()
y_preds

array([5., 4., 5., ..., 5., 5., 5.])

<IPython.core.display.Javascript object>

In [14]:
y_test = pd.DataFrame({"PassengerId": X_test.index, "CabinSeries": y_preds}).set_index(
    "PassengerId"
)
y_test

,CabinSeries
PassengerId,
1,5.0
3,4.0
5,5.0
6,5.0
8,5.0
...,...
1304,4.0
1305,5.0
1307,5.0


<IPython.core.display.Javascript object>

In [15]:
df["isCabinSeriesPreds"] = df["CabinSeries"].isna().astype(int)

<IPython.core.display.Javascript object>

In [16]:
df.fillna(y_test, inplace=True)
df["CabinSeries"] = df["CabinSeries"].astype(int)

<IPython.core.display.Javascript object>

In [17]:
df

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,...,Title,FamilySize,TicketSeries,NameSeries,SurnameSeries,TitleSeries,IsTest,isAgePreds,CabinSeries,isCabinSeriesPreds
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,0.0,3,Owen Harris,1,22.000000,1,0,A/5 21171,-0.503595,NaN,...,Mr,2,2,855,100,12,0,0,5,1
2,1.0,1,John Bradley (Florence Briggs Thayer),0,38.000000,1,0,PC 17599,0.734503,C85,...,Mrs,2,17,601,182,13,0,0,2,0
3,1.0,3,Laina,0,26.000000,0,0,STON/O2. 3101282,-0.490544,NaN,...,Miss,1,34,690,329,9,0,0,4,1
4,1.0,1,Jacques Heath (Lily May Peel),0,35.000000,1,0,113803,0.382925,C123,...,Mrs,2,40,541,267,13,0,0,2,0
5,0.0,3,William Henry,1,35.000000,0,0,373450,-0.488127,NaN,...,Mr,1,40,1102,15,12,0,0,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,NaN,3,Woolf,1,26.994279,0,0,A.5. 3236,-0.488127,NaN,...,Mr,1,5,1119,753,12,1,1,5,1
1306,NaN,1,Fermina,0,39.000000,0,0,PC 17758,1.461829,C105,...,Dona,1,17,366,593,3,1,0,2,0
1307,NaN,3,Simon Sivertsen,1,38.500000,0,0,SOTON/O.Q. 3101262,-0.503595,NaN,...,Mr,1,31,973,699,12,1,0,5,1


<IPython.core.display.Javascript object>

In [18]:
df.to_csv("../../data/titanic/prepare.csv")

<IPython.core.display.Javascript object>